### Credit Card Fraud Detection - Capstone Project
##### Upgrad - University of Arizona
##### Silambarasan Melapudi Kannappan

In [3]:
pip install -U pandasql

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 2.7 MB 44.7 MB/s eta 0:00:01
     |████████████████████████████████| 566 kB 52.2 MB/s eta 0:00:01
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26784 sha256=c03f61aeed23b4dbe2bf8b1f99796fc3c906ac234458cd0a326579d997f9c1f0
  Stored in directory: /home/emr-notebook/.cache/pip/wheels/5c/4b/ec/41f4e116c8053c3654e2c2a47c62b4fca34cc67ef7b55deb7f
Successfully built pandasql
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 24.8 MB 25.3 MB/s eta 0:00:01
     |████████████████████████████████| 38.1 MB 53.7 MB/s eta 0:00:01
     |████████████████████████████████| 297 kB 51.8 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install fsspec

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 143 kB 23.4 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install s3fs

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 66 kB 5.4 MB/s  eta 0:00:01
     |████████████████████████████████| 9.1 MB 55.4 MB/s eta 0:00:01
     |████████████████████████████████| 75 kB 7.3 MB/s  eta 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
boto3 1.9.66 requires botocore<1.13.0,>=1.12.66, but you have botocore 1.27.59 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import Statements
import pandas as pd
from pandasql import sqldf
import pandas as pd
from sklearn import datasets
import pymongo
from pymongo import MongoClient

### Task 1 : Load the transaction history data (card_transactions.csv) in a NoSQL database which is MongoDB 
#### (card_transactions.csv will be given to you - you only need to load the data in the MongoDB database)

Data is loaded in Mongo directly

### Task 2: Ingest the relevant data (card_member and member_score) from AWS RDS to Hadoop.

This task is done and given to you in form of CSV files named "card_member" and "member_score"}.

### Task 3: Create a look-up table in MongoDB {NoSQL database} with columns specified earlier in the problem statement.

In [3]:
# Load the csv files to pandas dataframe
df_cardMember = pd.read_csv('s3://aws-emr-resources-560518250500-us-east-1/Creditcard_frauddeduction/card_member.csv')
df_member_score = pd.read_csv('s3://aws-emr-resources-560518250500-us-east-1/Creditcard_frauddeduction/member_score.csv')
df_card_transactions = pd.read_csv('s3://aws-emr-resources-560518250500-us-east-1/Creditcard_frauddeduction/card_transactions.csv')

In [4]:
# check number of records in each dataframe
print("Number of Records in df_cardMember : ", df_cardMember.shape[0])
print("Number of Records in df_member_score is :", df_member_score.shape[0])
print("Number of Records in df_card_transactions is :", df_card_transactions.shape[0])

Number of Records in df_cardMember :  999
Number of Records in df_member_score is : 999
Number of Records in df_card_transactions is : 53292


In [5]:
df_cardMember

,card_id,member_id,member_joining_dt,card_purchase_dt,country,city
0,340028465709212,9250698176266,2012-02-08 06:04:13.0,05/13,United States,Barberton
1,340054675199675,835873341185231,2017-03-10 09:24:44.0,03/17,United States,Fort Dodge
2,340082915339645,512969555857346,2014-02-15 06:30:30.0,07/14,United States,Graham
3,340134186926007,887711945571282,2012-02-05 01:21:58.0,02/13,United States,Dix Hills
4,340265728490548,680324265406190,2014-03-29 07:49:14.0,11/14,United States,Rancho Cucamonga
...,...,...,...,...,...,...
994,6595814135833988,236864426408837,2011-02-18 06:45:46.0,06/14,United States,Johnston
995,6595928469079750,422173403853722,2012-02-10 02:19:20.0,02/14,United States,Raleigh
996,6597703848279563,657991752750272,2017-02-16 07:45:34.0,03/17,United States,Pinewood
997,6598830758632447,660350842993890,2013-07-26 02:22:17.0,12/16,United States,Tampa


In [6]:
df_member_score

,member_id,score
0,37495066290,339
1,117826301530,289
2,1147922084344,393
3,1314074991813,225
4,1739553947511,642
...,...,...
994,992104321998571,658
995,992552823055811,694
996,994983851226493,687
997,996411635289270,335


In [7]:
# take only genuine transactions into the dataframe
df_card_transactions = df_card_transactions[df_card_transactions.status == 'GENUINE']
df_card_transactions

,card_id,member_id,amount,postcode,pos_id,transaction_dt,status
0,348702330256514,37495066290,9084849,33946,614677375609919,11-02-2018 00:00:00,GENUINE
1,348702330256514,37495066290,330148,33946,614677375609919,11-02-2018 00:00:00,GENUINE
2,348702330256514,37495066290,136052,33946,614677375609919,11-02-2018 00:00:00,GENUINE
3,348702330256514,37495066290,4310362,33946,614677375609919,11-02-2018 00:00:00,GENUINE
4,348702330256514,37495066290,9097094,33946,614677375609919,11-02-2018 00:00:00,GENUINE
...,...,...,...,...,...,...,...
53287,5391723993945313,997128952368160,7234353,54514,366418882890893,30-01-2018 14:21:15,GENUINE
53288,5391723993945313,997128952368160,283051,20711,479856487174409,30-11-2017 05:24:26,GENUINE
53289,5391723993945313,997128952368160,5527334,20711,479856487174409,30-11-2017 07:09:46,GENUINE
53290,5391723993945313,997128952368160,714092,20711,479856487174409,30-11-2017 12:25:47,GENUINE


In [8]:
df_card_transactions.dtypes

card_id            int64
member_id          int64
amount             int64
postcode           int64
pos_id             int64
transaction_dt    object
status            object
dtype: object

In [9]:
#convert from object to datetime
df_card_transactions['transaction_dt2']=pd.to_datetime(df_card_transactions['transaction_dt'])

In [10]:
df_card_transactions.dtypes

card_id                     int64
member_id                   int64
amount                      int64
postcode                    int64
pos_id                      int64
transaction_dt             object
status                     object
transaction_dt2    datetime64[ns]
dtype: object

In [11]:
# Sort descending order
df_card_transactions.sort_values(by=['member_id','transaction_dt2'], inplace=True,
               ascending = [True, False])
df_card_transactions

,card_id,member_id,amount,postcode,pos_id,transaction_dt,status,transaction_dt2
0,348702330256514,37495066290,9084849,33946,614677375609919,11-02-2018 00:00:00,GENUINE,2018-11-02 00:00:00
1,348702330256514,37495066290,330148,33946,614677375609919,11-02-2018 00:00:00,GENUINE,2018-11-02 00:00:00
2,348702330256514,37495066290,136052,33946,614677375609919,11-02-2018 00:00:00,GENUINE,2018-11-02 00:00:00
3,348702330256514,37495066290,4310362,33946,614677375609919,11-02-2018 00:00:00,GENUINE,2018-11-02 00:00:00
4,348702330256514,37495066290,9097094,33946,614677375609919,11-02-2018 00:00:00,GENUINE,2018-11-02 00:00:00
...,...,...,...,...,...,...,...,...
53207,5391723993945313,997128952368160,1396526,53968,716276786938354,05-11-2017 12:00:54,GENUINE,2017-05-11 12:00:54
53206,5391723993945313,997128952368160,25146,13124,200432834928149,04-11-2017 18:27:33,GENUINE,2017-04-11 18:27:33
53205,5391723993945313,997128952368160,2727621,57376,97278983752405,03-12-2017 22:56:34,GENUINE,2017-03-12 22:56:34
53204,5391723993945313,997128952368160,7201018,48832,712082643297946,03-11-2017 19:38:11,GENUINE,2017-03-11 19:38:11


In [12]:
#Group by card_id, date in descending order and create a rank column
# keep only top 10 rank for UCL calcuation
df_card_transactions["Rank"] = df_card_transactions.groupby("card_id")["transaction_dt2"].rank(ascending=False, method='dense').astype(int)
df_card_transactions.sort_values("Rank")
df_card_transactions_top10 = df_card_transactions[(df_card_transactions.Rank <=10) ]
df_card_transactions_top10

,card_id,member_id,amount,postcode,pos_id,transaction_dt,status,transaction_dt2,Rank
0,348702330256514,37495066290,9084849,33946,614677375609919,11-02-2018 00:00:00,GENUINE,2018-11-02 00:00:00,1
1,348702330256514,37495066290,330148,33946,614677375609919,11-02-2018 00:00:00,GENUINE,2018-11-02 00:00:00,1
2,348702330256514,37495066290,136052,33946,614677375609919,11-02-2018 00:00:00,GENUINE,2018-11-02 00:00:00,1
3,348702330256514,37495066290,4310362,33946,614677375609919,11-02-2018 00:00:00,GENUINE,2018-11-02 00:00:00,1
4,348702330256514,37495066290,9097094,33946,614677375609919,11-02-2018 00:00:00,GENUINE,2018-11-02 00:00:00,1
...,...,...,...,...,...,...,...,...,...
53202,5391723993945313,997128952368160,9670623,77974,675701087475560,03-01-2018 07:30:58,GENUINE,2018-03-01 07:30:58,6
53200,5391723993945313,997128952368160,1164872,93210,171173202259233,02-01-2018 19:13:37,GENUINE,2018-02-01 19:13:37,7
53287,5391723993945313,997128952368160,7234353,54514,366418882890893,30-01-2018 14:21:15,GENUINE,2018-01-30 14:21:15,8
53284,5391723993945313,997128952368160,2683995,44065,42771743906415,29-01-2018 13:46:33,GENUINE,2018-01-29 13:46:33,9


In [13]:
#Check if only genuine transactions exist
df_card_transactions_top10.groupby('status')['status'].value_counts()

status   status 
GENUINE  GENUINE    10953
Name: status, dtype: int64

In [14]:
# calculate mean and standard deviation on amount for each member
df_ucl_sd = df_card_transactions_top10.groupby(['member_id'])['amount'].agg(['mean', 'std'])
df_ucl_sd

,mean,std
member_id,,
37495066290,4.260123e+06,2.917135e+06
117826301530,4.300037e+06,2.666817e+06
1147922084344,3.442887e+06,2.816608e+06
1314074991813,4.937506e+06,2.618911e+06
1739553947511,5.040048e+06,3.333798e+06
...,...,...
992104321998571,6.906752e+06,2.194694e+06
992552823055811,4.959108e+06,3.389936e+06
994983851226493,5.433286e+06,2.733345e+06


In [15]:
# reset index to make member_id a column
df_ucl_sd = df_ucl_sd.reset_index()
df_ucl_sd

,member_id,mean,std
0,37495066290,4.260123e+06,2.917135e+06
1,117826301530,4.300037e+06,2.666817e+06
2,1147922084344,3.442887e+06,2.816608e+06
3,1314074991813,4.937506e+06,2.618911e+06
4,1739553947511,5.040048e+06,3.333798e+06
...,...,...,...
994,992104321998571,6.906752e+06,2.194694e+06
995,992552823055811,4.959108e+06,3.389936e+06
996,994983851226493,5.433286e+06,2.733345e+06
997,996411635289270,5.629739e+06,3.168106e+06


In [16]:
# check datatypes on the dataframe
df_ucl_sd.dtypes

member_id      int64
mean         float64
std          float64
dtype: object

In [17]:
# rename columns
df_ucl_sd.rename(columns = {'mean':'Avg1', 'std':'SD'}, inplace = True)


In [18]:
# calculate UCL with below formula
df_ucl_sd['ucl'] = df_ucl_sd.Avg1 +  3*df_ucl_sd.SD

In [19]:
df_ucl_sd

,member_id,Avg1,SD,ucl
0,37495066290,4.260123e+06,2.917135e+06,1.301153e+07
1,117826301530,4.300037e+06,2.666817e+06,1.230049e+07
2,1147922084344,3.442887e+06,2.816608e+06,1.189271e+07
3,1314074991813,4.937506e+06,2.618911e+06,1.279424e+07
4,1739553947511,5.040048e+06,3.333798e+06,1.504144e+07
...,...,...,...,...
994,992104321998571,6.906752e+06,2.194694e+06,1.349084e+07
995,992552823055811,4.959108e+06,3.389936e+06,1.512892e+07
996,994983851226493,5.433286e+06,2.733345e+06,1.363332e+07
997,996411635289270,5.629739e+06,3.168106e+06,1.513406e+07


In [20]:
# keep only rank 1 for furthe processing 
lstTxnPostCodeTxDate = df_card_transactions[df_card_transactions.Rank ==1]
lstTxnPostCodeTxDate

,card_id,member_id,amount,postcode,pos_id,transaction_dt,status,transaction_dt2,Rank
0,348702330256514,37495066290,9084849,33946,614677375609919,11-02-2018 00:00:00,GENUINE,2018-11-02 00:00:00,1
1,348702330256514,37495066290,330148,33946,614677375609919,11-02-2018 00:00:00,GENUINE,2018-11-02 00:00:00,1
2,348702330256514,37495066290,136052,33946,614677375609919,11-02-2018 00:00:00,GENUINE,2018-11-02 00:00:00,1
3,348702330256514,37495066290,4310362,33946,614677375609919,11-02-2018 00:00:00,GENUINE,2018-11-02 00:00:00,1
4,348702330256514,37495066290,9097094,33946,614677375609919,11-02-2018 00:00:00,GENUINE,2018-11-02 00:00:00,1
...,...,...,...,...,...,...,...,...,...
53046,5115273575078211,992104321998571,8416500,42082,609717185246135,08-01-2018 15:40:29,GENUINE,2018-08-01 15:40:29,1
53097,374878208630443,992552823055811,4185444,12747,640304888744272,04-01-2018 22:08:21,GENUINE,2018-04-01 22:08:21,1
53162,5399715069800093,994983851226493,4302555,77432,487096286569669,30-01-2018 09:05:15,GENUINE,2018-01-30 09:05:15,1
53189,4981548424137027,996411635289270,4993560,92694,181718433544561,18-12-2017 00:01:04,GENUINE,2017-12-18 00:01:04,1


In [21]:
# merge dataframes to get other columns - transaction date, postcode, ucl
tmp1 = df_ucl_sd.merge(lstTxnPostCodeTxDate, on='member_id', how='left')
tmp1

,member_id,Avg1,SD,ucl,card_id,amount,postcode,pos_id,transaction_dt,status,transaction_dt2,Rank
0,37495066290,4.260123e+06,2.917135e+06,1.301153e+07,348702330256514,9084849,33946,614677375609919,11-02-2018 00:00:00,GENUINE,2018-11-02 00:00:00,1
1,37495066290,4.260123e+06,2.917135e+06,1.301153e+07,348702330256514,330148,33946,614677375609919,11-02-2018 00:00:00,GENUINE,2018-11-02 00:00:00,1
2,37495066290,4.260123e+06,2.917135e+06,1.301153e+07,348702330256514,136052,33946,614677375609919,11-02-2018 00:00:00,GENUINE,2018-11-02 00:00:00,1
3,37495066290,4.260123e+06,2.917135e+06,1.301153e+07,348702330256514,4310362,33946,614677375609919,11-02-2018 00:00:00,GENUINE,2018-11-02 00:00:00,1
4,37495066290,4.260123e+06,2.917135e+06,1.301153e+07,348702330256514,9097094,33946,614677375609919,11-02-2018 00:00:00,GENUINE,2018-11-02 00:00:00,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1653,992104321998571,6.906752e+06,2.194694e+06,1.349084e+07,5115273575078211,8416500,42082,609717185246135,08-01-2018 15:40:29,GENUINE,2018-08-01 15:40:29,1
1654,992552823055811,4.959108e+06,3.389936e+06,1.512892e+07,374878208630443,4185444,12747,640304888744272,04-01-2018 22:08:21,GENUINE,2018-04-01 22:08:21,1
1655,994983851226493,5.433286e+06,2.733345e+06,1.363332e+07,5399715069800093,4302555,77432,487096286569669,30-01-2018 09:05:15,GENUINE,2018-01-30 09:05:15,1
1656,996411635289270,5.629739e+06,3.168106e+06,1.513406e+07,4981548424137027,4993560,92694,181718433544561,18-12-2017 00:01:04,GENUINE,2017-12-18 00:01:04,1


In [22]:
# merge dataframe to get score
tmp2 = df_member_score.merge(tmp1, on='member_id', how='left')
tmp2

,member_id,score,Avg1,SD,ucl,card_id,amount,postcode,pos_id,transaction_dt,status,transaction_dt2,Rank
0,37495066290,339,4.260123e+06,2.917135e+06,1.301153e+07,348702330256514,9084849,33946,614677375609919,11-02-2018 00:00:00,GENUINE,2018-11-02 00:00:00,1
1,37495066290,339,4.260123e+06,2.917135e+06,1.301153e+07,348702330256514,330148,33946,614677375609919,11-02-2018 00:00:00,GENUINE,2018-11-02 00:00:00,1
2,37495066290,339,4.260123e+06,2.917135e+06,1.301153e+07,348702330256514,136052,33946,614677375609919,11-02-2018 00:00:00,GENUINE,2018-11-02 00:00:00,1
3,37495066290,339,4.260123e+06,2.917135e+06,1.301153e+07,348702330256514,4310362,33946,614677375609919,11-02-2018 00:00:00,GENUINE,2018-11-02 00:00:00,1
4,37495066290,339,4.260123e+06,2.917135e+06,1.301153e+07,348702330256514,9097094,33946,614677375609919,11-02-2018 00:00:00,GENUINE,2018-11-02 00:00:00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1653,992104321998571,658,6.906752e+06,2.194694e+06,1.349084e+07,5115273575078211,8416500,42082,609717185246135,08-01-2018 15:40:29,GENUINE,2018-08-01 15:40:29,1
1654,992552823055811,694,4.959108e+06,3.389936e+06,1.512892e+07,374878208630443,4185444,12747,640304888744272,04-01-2018 22:08:21,GENUINE,2018-04-01 22:08:21,1
1655,994983851226493,687,5.433286e+06,2.733345e+06,1.363332e+07,5399715069800093,4302555,77432,487096286569669,30-01-2018 09:05:15,GENUINE,2018-01-30 09:05:15,1
1656,996411635289270,335,5.629739e+06,3.168106e+06,1.513406e+07,4981548424137027,4993560,92694,181718433544561,18-12-2017 00:01:04,GENUINE,2017-12-18 00:01:04,1


In [23]:
# drop the redundant columns
columns = ['amount', 'pos_id', 'status', 'transaction_dt', 'status', 'Rank', 'Avg1', 'SD']
tmp2.drop(columns, inplace=True, axis=1)
lookup_df = tmp2
lookup_df = lookup_df.drop_duplicates(subset=['member_id'])
lookup_df = lookup_df.reset_index()
lookup_df

,index,member_id,score,ucl,card_id,postcode,transaction_dt2
0,0,37495066290,339,1.301153e+07,348702330256514,33946,2018-11-02 00:00:00
1,42,117826301530,289,1.230049e+07,5189563368503974,30234,2018-06-01 09:15:04
2,43,1147922084344,393,1.189271e+07,5407073344486464,63770,2018-01-18 10:55:30
3,44,1314074991813,225,1.279424e+07,378303738095292,41778,2018-06-01 14:31:05
4,45,1739553947511,642,1.504144e+07,348413196172048,92258,2018-12-01 21:29:58
...,...,...,...,...,...,...,...
994,1653,992104321998571,658,1.349084e+07,5115273575078211,42082,2018-08-01 15:40:29
995,1654,992552823055811,694,1.512892e+07,374878208630443,12747,2018-04-01 22:08:21
996,1655,994983851226493,687,1.363332e+07,5399715069800093,77432,2018-01-30 09:05:15
997,1656,996411635289270,335,1.513406e+07,4981548424137027,92694,2017-12-18 00:01:04


In [24]:
#final lookup dataframe
final_lookup_df = lookup_df[['member_id','score','ucl','card_id','postcode','transaction_dt2']]
final_lookup_df

,member_id,score,ucl,card_id,postcode,transaction_dt2
0,37495066290,339,1.301153e+07,348702330256514,33946,2018-11-02 00:00:00
1,117826301530,289,1.230049e+07,5189563368503974,30234,2018-06-01 09:15:04
2,1147922084344,393,1.189271e+07,5407073344486464,63770,2018-01-18 10:55:30
3,1314074991813,225,1.279424e+07,378303738095292,41778,2018-06-01 14:31:05
4,1739553947511,642,1.504144e+07,348413196172048,92258,2018-12-01 21:29:58
...,...,...,...,...,...,...
994,992104321998571,658,1.349084e+07,5115273575078211,42082,2018-08-01 15:40:29
995,992552823055811,694,1.512892e+07,374878208630443,12747,2018-04-01 22:08:21
996,994983851226493,687,1.363332e+07,5399715069800093,77432,2018-01-30 09:05:15
997,996411635289270,335,1.513406e+07,4981548424137027,92694,2017-12-18 00:01:04


In [25]:
#saving the file to csv ; this file is loaded into mongodb
#determining the name of the file
file_name = 'lookupTable4.csv'

#saving the excel
final_lookup_df.to_csv(file_name) 
print('DataFrame lookup_df - is written to Excel File successfully.')

PermissionError: [Errno 13] Permission denied: 'lookupTable4.csv'

In [26]:
# Connect to MongoDB
client =  MongoClient("mongodb://localhost:27017")
#MongoClient("mongodb+srv://<<YOUR USERNAME>>:<<PASSWORD>>@clustertest-icsum.mongodb.net/test?retryWrites=true&w=majority")
db = client['CREDIT_CARD_DB']
collection = db['lookup_table']
#final_lookup_df.reset_index(inplace=True)
data_dict = final_lookup_df.to_dict("records")

# Insert collection

if collection.count_documents({})==0:
    for i in range(len(data_dict)):
        collection.insert_one(data_dict[i])
    print("Lookup table has been updated")
else:
    collection.drop()
    for i in range(len(data_dict)):
        collection.insert_one(data_dict[i])
    print("Lookup table has been updated")

ServerSelectionTimeoutError: localhost:27017: [Errno 111] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 63eaa0b9e0c37555db7c74f3, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused')>]>